<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/Bioco_Mw_Semiveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna
!pip install LAMDA-SSL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection
from statistics import mean

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

Evalueringsmetrikker

In [4]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_faktisk, y_predikert):
  return np.sqrt(mean_squared_error(y_faktisk, y_predikert))

# Egendefinerte moduler

In [5]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [6]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import umarkert_andel_enzym
from oppdeling_trening_testsett import umarkert_andel_dag
from oppdeling_trening_testsett import enzymfordeling

In [7]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_semiveiledet.py /content/moduler_optimalisering_evaluering_semiveiledet.py

/content


In [8]:
from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_enzym

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_dag

# Importering av relevant data

In [9]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                   header=0,
                   sep=',',
                   index_col=0)

# Normal produksjon
data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_normal.csv',
                          header=0,
                          sep=',',
                          index_col=0)

data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_design.csv',
                          header=0,
                          sep=',',
                          index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_normal.index = pd.to_datetime(data_normal.index,
                                   format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [10]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29049 entries, 2022-10-31 17:37:00 to 2023-06-14 01:05:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29049 non-null  int64  
 1   EnzymeType_A2  29049 non-null  int64  
 2   EnzymeType_B   29049 non-null  int64  
 3   EnzymeType_C   29049 non-null  int64  
 4   EnzymeType_D   29049 non-null  int64  
 5   EnzymeType_E   29049 non-null  int64  
 6   RawMatPercent  29049 non-null  float64
 7   NIRfat         29049 non-null  float64
 8   NIRash         29049 non-null  float64
 9   NIRwater       29049 non-null  float64
 10  TT08           29049 non-null  float64
 11  TT20           29049 non-null  float64
 12  TT12           29049 non-null  float64
 13  Mw             460 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [12]:
data_normal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18542 entries, 2022-11-02 00:00:00 to 2023-06-14 01:05:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  18542 non-null  int64  
 1   EnzymeType_A2  18542 non-null  int64  
 2   EnzymeType_B   18542 non-null  int64  
 3   EnzymeType_C   18542 non-null  int64  
 4   EnzymeType_D   18542 non-null  int64  
 5   EnzymeType_E   18542 non-null  int64  
 6   RawMatPercent  18542 non-null  float64
 7   NIRfat         18542 non-null  float64
 8   NIRash         18542 non-null  float64
 9   NIRwater       18542 non-null  float64
 10  TT08           18542 non-null  float64
 11  TT20           18542 non-null  float64
 12  TT12           18542 non-null  float64
 13  Mw             124 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 2.1 MB


In [13]:
data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10507 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10507 non-null  int64  
 1   EnzymeType_A2  10507 non-null  int64  
 2   EnzymeType_B   10507 non-null  int64  
 3   EnzymeType_C   10507 non-null  int64  
 4   EnzymeType_D   10507 non-null  int64  
 5   EnzymeType_E   10507 non-null  int64  
 6   RawMatPercent  10507 non-null  float64
 7   NIRfat         10507 non-null  float64
 8   NIRash         10507 non-null  float64
 9   NIRwater       10507 non-null  float64
 10  TT08           10507 non-null  float64
 11  TT20           10507 non-null  float64
 12  TT12           10507 non-null  float64
 13  Mw             336 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# Lagring av resultater

In [16]:
# Kolonner til dataframe for lagring av resultater fra opmtimalisering
resultater_kolonner = ["Treningsandel",
                       "RMSE test",
                       "R2 test",
                       "MAE test",
                       "MAPE test",
                       "RMSE trening",
                       "R2 trening",
                       "MAE trening",
                       "MAPE trening"]

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [17]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

# Definerer testandel
test_andel = 0.2

# Definere andel av treningsett som skal benyttes
trenings_andel = 1.0


# Deler datasettet i trening og testsett
treningssett_markert, enzymtyper_treningssett_markert, testsett,\
emzymtyper_testsett,treningssett_umarkert, enzymtyper_treningssett_umarkert,\
treningssett =trening_testsett_oppdeling_enzym(rå_data,
                                               data,
                                               test_andel=test_andel)


#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_treningssett_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(emzymtyper_testsett.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_treningssett_umarkert.value_counts())



# Treningssett i forklaringsvariabler og respons
X_test = testsett.iloc[:, :-1]
y_test = testsett.iloc[:, -1]

# Umarkert andel til BDH
umarkert_andel = 0.001

# Kryssvalidering
splitt = 10
gjentagelser = 10

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
A2    128
E      60
C      56
B      56
D      50
A1     18
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
A2    32
E     15
C     14
B     14
D     13
A1     4
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
A2    17987
A1     2823
D      2193
B      2043
C      1846
E      1697
Name: EnzymeCode, dtype: int64


### CoReg

In [16]:
# Modelloptimalisering
coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                   enzymtyper_treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt=splitt,
                                                   gjentagelser=gjentagelser,
                                                   n_runder=n_runder,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


# Legger resulater til i datasett
coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-22 16:27:06,158] A new study created in memory with name: no-name-e2ddb2f9-fd1d-4fc1-9f77-0047316a0e2a
[W 2024-03-22 16:27:41,884] Trial 0 failed with parameters: {'k1': 8, 'k2': 8, 'p1': 4, 'p2': 9, 'max_iters': 10, 'pool_size': 108} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/content/moduler_optimalisering_evaluering_semiveiledet.py", line 277, in objective
    coreg.fit(X_trening_fold_markert_transformert,
  File "/usr/local/lib/python3.10/dist-packages/LAMDA_SSL/Algorithm/Regression/CoReg.py", line 78, in fit
    delta -= (L_y[idx_o].reshape(1, -1) -
KeyboardInterrupt
[W 2024-03-22 16:27:41,890] Trial 0 failed with value None.


KeyboardInterrupt: 

### BHD

In [18]:
# BHD
bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                               enzymtyper_treningssett_markert,
                                               treningssett_umarkert,
                                               enzymtyper_treningssett_umarkert,
                                               treningssett,
                                               testsett,
                                               X_test,
                                               y_test,
                                               splitt,
                                               gjentagelser,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


# Legger resulater til i datasett
bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-22 17:24:57,954] A new study created in memory with name: no-name-d221bbaf-db11-44e4-9324-01951d0cdef3
[I 2024-03-22 17:25:11,356] Trial 0 finished with value: 1784.3210773239907 and parameters: {'k': 199, 'alpha': 0.39062349423348086, 'maks_iterasjoner': 91}. Best is trial 0 with value: 1784.3210773239907.
[I 2024-03-22 17:25:25,826] Trial 1 finished with value: 1935.4035935156082 and parameters: {'k': 281, 'alpha': 0.26757885188897346, 'maks_iterasjoner': 199}. Best is trial 0 with value: 1784.3210773239907.
[I 2024-03-22 17:25:39,026] Trial 2 finished with value: 1495.5032832378047 and parameters: {'k': 206, 'alpha': 0.7514721418956966, 'maks_iterasjoner': 67}. Best is trial 2 with value: 1495.5032832378047.
[I 2024-03-22 17:25:54,946] Trial 3 finished with value: 1732.3521876636873 and parameters: {'k': 375, 'alpha': 0.43973935559749977, 'maks_iterasjoner': 68}. Best is trial 2 with value: 1495.5032832378047.
[I 2024-03-22 17:26:11,034] Trial 4 finished with value: 1676.

{'k': 353, 'alpha': 0.9435109348442233, 'maks_iterasjoner': 126}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,1358.679867,0.67235,1114.672916,0.170962,360.141378,0.977491,130.738192,0.020755


## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzymtyper_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_umarkert.value_counts())

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.5, 0.7]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 1

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
A2    7
C     4
B     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())

  # Modelloptimalisering
  coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                     enzymtyper_treningssett_markert,
                                                     treningssett_umarkert,
                                                     treningssett,
                                                     X_test,
                                                     y_test,
                                                     splitt=splitt,
                                                     gjentagelser=gjentagelser,
                                                     n_runder=n_runder,
                                                     trenings_andel=trenings_andel,
                                                     random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-19 19:10:16,250] A new study created in memory with name: no-name-d5082504-b0e3-4491-a23f-f400ce3cfb7c


Enzymtyper Fordeling: Treningssett markert andel: 0.5
A2    13
C      8
B      8
D      5
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-19 19:10:23,691] Trial 0 finished with value: 4.4196957334928495 and parameters: {'k1': 9, 'k2': 4, 'p1': 10, 'p2': 9, 'max_iters': 19, 'pool_size': 14}. Best is trial 0 with value: 4.4196957334928495.
[I 2024-03-19 19:10:27,655] A new study created in memory with name: no-name-0bc4fce8-9453-4093-b10b-035d8a0e8352


Enzymtyper Fordeling: Treningssett markert andel: 0.7
A2    18
C     11
B     11
D      8
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-19 19:10:31,232] Trial 0 finished with value: 4.9286611096899415 and parameters: {'k1': 5, 'k2': 1, 'p1': 3, 'p2': 5, 'max_iters': 1, 'pool_size': 154}. Best is trial 0 with value: 4.9286611096899415.
[I 2024-03-19 19:10:32,613] A new study created in memory with name: no-name-ee9b2a82-2957-4996-bcb2-901959c8fd9d


Enzymtyper Fordeling: Treningssett markert andel: 0.9
A2    23
B     14
C     14
D     10
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-19 19:10:39,398] Trial 0 finished with value: 3.9044307959768414 and parameters: {'k1': 6, 'k2': 8, 'p1': 4, 'p2': 8, 'max_iters': 18, 'pool_size': 18}. Best is trial 0 with value: 3.9044307959768414.


{'trenings_andel': [0.5, 0.7, 0.9], 'parametere': [{'k1': 9, 'k2': 4, 'p1': 10, 'p2': 9, 'max_iters': 19, 'pool_size': 14}, {'k1': 5, 'k2': 1, 'p1': 3, 'p2': 5, 'max_iters': 1, 'pool_size': 154}, {'k1': 6, 'k2': 8, 'p1': 4, 'p2': 8, 'max_iters': 18, 'pool_size': 18}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.5,3.808039,0.582720,2.817819,0.152346,2.697001,0.666389,2.147705,0.089039
1,0.7,4.614026,0.387388,3.850733,0.201283,1.555246,0.895427,1.250396,0.057692
2,0.9,4.171275,0.499317,3.346993,0.178940,2.965527,0.651220,2.376538,0.112246


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # BHD
  bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 enzymtyper_umarkert,
                                                 treningssett,
                                                 test,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 gjentagelser,
                                                 n_runder,
                                                 umarkert_andel=umarkert_andel,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-19 20:10:25,479] A new study created in memory with name: no-name-7c6a7a85-f538-40f0-9cc2-71a63dd70649


Enzymtyper Fordeling: Treningssett markert andel: 0.5
A2    13
C      8
B      8
D      5
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-19 20:11:25,482] Trial 0 finished with value: 5.463257439018376 and parameters: {'k': 6520, 'alpha': 0.01, 'maks_iterasjoner': 59}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:12:20,756] Trial 1 finished with value: 9.440781297034448 and parameters: {'k': 6420, 'alpha': 1, 'maks_iterasjoner': 21}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:13:09,686] Trial 2 finished with value: 13.284018048511529 and parameters: {'k': 3291, 'alpha': 10, 'maks_iterasjoner': 41}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:14:03,260] Trial 3 finished with value: 9.38236450932272 and parameters: {'k': 5615, 'alpha': 1, 'maks_iterasjoner': 67}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:14:24,211] Trial 4 finished with value: 13.283227581124137 and parameters: {'k': 608, 'alpha': 10, 'maks_iterasjoner': 61}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:15:20,603] Trial 5 finished with value: 9.4

Enzymtyper Fordeling: Treningssett markert andel: 0.7
A2    18
C     11
B     11
D      8
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-19 20:21:47,669] Trial 0 finished with value: 5.170170181366991 and parameters: {'k': 6810, 'alpha': 0.1, 'maks_iterasjoner': 124}. Best is trial 0 with value: 5.170170181366991.
[I 2024-03-19 20:22:46,339] Trial 1 finished with value: 5.170124841396653 and parameters: {'k': 2400, 'alpha': 0.1, 'maks_iterasjoner': 175}. Best is trial 1 with value: 5.170124841396653.
[I 2024-03-19 20:23:47,677] Trial 2 finished with value: 5.1702718989745815 and parameters: {'k': 6635, 'alpha': 0.1, 'maks_iterasjoner': 75}. Best is trial 1 with value: 5.170124841396653.
[I 2024-03-19 20:24:27,375] Trial 3 finished with value: 9.511713575788109 and parameters: {'k': 1496, 'alpha': 1, 'maks_iterasjoner': 142}. Best is trial 1 with value: 5.170124841396653.
[I 2024-03-19 20:24:57,188] Trial 4 finished with value: 25.524638276899225 and parameters: {'k': 1557, 'alpha': 10, 'maks_iterasjoner': 8}. Best is trial 1 with value: 5.170124841396653.
[I 2024-03-19 20:25:52,165] Trial 5 finished with valu

{'trenings_andel': [0.5, 0.7], 'parametere': [{'k': 2851, 'alpha': 0.01, 'maks_iterasjoner': 177}, {'k': 931, 'alpha': 0.01, 'maks_iterasjoner': 198}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,6.082883,-0.064742,4.513696,0.247507,0.126221,0.999269,0.093372,0.003768
1,1.0,6.111859,-0.074909,4.537162,0.249573,0.132864,0.999237,0.100209,0.004191


## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [20]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_trening_markert = enzymfordeling(rå_data,
                                       trening_markert)
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)

print("Enzymtyper Fordeling: Trening markert")
print(enzym_trening_markert)
print("Enzymtyper Fordeling: Test")
print(enzym_test)
print("Enzymtyper Fordeling: Trening umarkert")
print(enzym_trening_umarkert)


# umarkert andel til BHD
umarkert_andel = 0.001

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Kryssvalidering
splitt = 10

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
    Frekvens  Andel
A2       134  36.31
E         60  16.26
C         55  14.91
B         54  14.63
D         50  13.55
A1        16   4.34
Enzymtyper Fordeling: Test
    Frekvens  Andel
A2        26  28.57
B         16  17.58
C         15  16.48
E         15  16.48
D         13  14.29
A1         6   6.59
Enzymtyper Fordeling: Trening umarkert
    Frekvens  Andel
A2     17987  62.92
A1      2823   9.87
D       2193   7.67
B       2043   7.15
C       1846   6.46
E       1697   5.94


### CoReg

In [16]:
# CoReg
coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                 trening_markert,
                                                 trening_umarkert,
                                                 trening,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 n_runder,
                                                 umarkert_andel=False,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


# Legger resulater til i datasett
coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-21 15:22:18,276] A new study created in memory with name: no-name-2da60f83-632f-45d3-be1b-21b834b0b9de
[I 2024-03-21 15:22:31,084] Trial 0 finished with value: 726.185803837625 and parameters: {'k1': 9, 'k2': 3, 'p1': 7, 'p2': 9, 'max_iters': 8, 'pool_size': 36}. Best is trial 0 with value: 726.185803837625.
[I 2024-03-21 15:22:53,198] Trial 1 finished with value: 711.524296708141 and parameters: {'k1': 3, 'k2': 4, 'p1': 6, 'p2': 5, 'max_iters': 6, 'pool_size': 96}. Best is trial 1 with value: 711.524296708141.


{'k1': 3, 'k2': 4, 'p1': 6, 'p2': 5, 'max_iters': 6, 'pool_size': 96}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,671.857479,0.912271,475.802992,0.066626,392.306065,0.973682,269.845695,0.040333


### BHD

In [21]:
# BHD
bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             test,
                                             X_test,
                                             y_test,
                                             splitt,
                                             n_runder,
                                             umarkert_andel=umarkert_andel,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


# Legger resulater til i datasett
bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

[I 2024-03-22 17:27:53,198] A new study created in memory with name: no-name-fdf74855-9d09-46fd-8c15-e31d8b63f27f
[I 2024-03-22 17:28:00,222] Trial 0 finished with value: 1841.4819370841565 and parameters: {'k': 372, 'alpha': 0.3022894928863835, 'maks_iterasjoner': 37}. Best is trial 0 with value: 1841.4819370841565.
[I 2024-03-22 17:28:08,072] Trial 1 finished with value: 1589.7015362979391 and parameters: {'k': 350, 'alpha': 0.7751593239857696, 'maks_iterasjoner': 38}. Best is trial 1 with value: 1589.7015362979391.
[I 2024-03-22 17:28:16,439] Trial 2 finished with value: 1912.0977852968322 and parameters: {'k': 363, 'alpha': 0.21815852294589422, 'maks_iterasjoner': 3}. Best is trial 1 with value: 1589.7015362979391.
[I 2024-03-22 17:28:22,959] Trial 3 finished with value: 1615.27064406949 and parameters: {'k': 290, 'alpha': 0.6980918114289718, 'maks_iterasjoner': 179}. Best is trial 1 with value: 1589.7015362979391.
[I 2024-03-22 17:28:30,962] Trial 4 finished with value: 1733.70287

{'k': 242, 'alpha': 0.9457862015172624, 'maks_iterasjoner': 62}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,3028.309718,-0.782335,2305.045236,0.301234,248.786824,0.989416,61.252243,0.011203


## Alternativ 2: Flere treningssandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                          'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data,
                                                           test_andel=test_andel,
                                                           trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)



print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Treningssett umarkert")
print(enzym_trening_umarkert.value_counts())


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
A2    26
B     16
C     15
E     15
D     13
A1     6
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert
A2    17987
A1     2823
D      2193
B      2043
C      1846
E      1697
Name: EnzymeCode, dtype: int64


### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # CoReg
  coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                   treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt,
                                                   n_runder,
                                                   umarkert_andel=False,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 15:27:13,528] A new study created in memory with name: no-name-77d34647-443e-4bcb-b3ae-3ccce255c618


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    134
E      60
C      55
B      54
D      50
A1     16
Name: EnzymeCode, dtype: int64


[I 2024-03-20 15:27:34,253] Trial 0 finished with value: 703.1147383611417 and parameters: {'k1': 5, 'k2': 3, 'p1': 6, 'p2': 2, 'max_iters': 5, 'pool_size': 181}. Best is trial 0 with value: 703.1147383611417.
[I 2024-03-20 15:27:48,255] Trial 1 finished with value: 737.5078416020515 and parameters: {'k1': 7, 'k2': 9, 'p1': 2, 'p2': 2, 'max_iters': 2, 'pool_size': 188}. Best is trial 0 with value: 703.1147383611417.


{'trenings_andel': [1.0], 'parametere': [{'k1': 5, 'k2': 3, 'p1': 6, 'p2': 2, 'max_iters': 5, 'pool_size': 181}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,672.422568,0.912123,474.244982,0.065673,397.4104,0.972992,272.138045,0.040787


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # BHD
  bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                               treningssett_markert,
                                               treningssett_umarkert,
                                               test,
                                               X_test,
                                               y_test,
                                               splitt,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    134
E      60
C      55
B      54
D      50
A1     16
Name: EnzymeCode, dtype: int64


[I 2024-03-20 15:29:39,565] A new study created in memory with name: no-name-23a6465d-2d7f-45d9-8c0e-6ba0873cbb53
[I 2024-03-20 15:30:04,597] Trial 0 finished with value: 1.1451791792705249e+18 and parameters: {'k': 681, 'alpha': 0.1, 'maks_iterasjoner': 82}. Best is trial 0 with value: 1.1451791792705249e+18.
[I 2024-03-20 15:31:14,147] Trial 1 finished with value: 3512.093901806471 and parameters: {'k': 4832, 'alpha': 1, 'maks_iterasjoner': 158}. Best is trial 1 with value: 3512.093901806471.


{'trenings_andel': [1.0], 'parametere': [{'k': 4832, 'alpha': 1, 'maks_iterasjoner': 158}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,3271.078441,-1.079556,2677.910522,0.353199,3342.692043,-0.910737,2545.770197,0.372238


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [76]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzymtyper_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_umarkert.value_counts())

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Umarkert andel til BDH
umarkert_andel = 0.05

# Kryssvalidering
splitt = 5
gjentagelser = 10

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
E     60
C     53
B     52
D     48
A2    41
A1    14
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
E     15
C     14
B     13
D     12
A2    10
A1     4
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
D     1926
A1    1826
B     1728
C     1710
E     1631
A2    1350
Name: EnzymeCode, dtype: int64


### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # Modelloptimalisering
  coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                    enzymtyper_treningssett_markert,
                                                    treningssett_umarkert,
                                                    treningssett,
                                                    X_test,
                                                    y_test,
                                                    splitt=splitt,
                                                    gjentagelser=gjentagelser,
                                                    n_runder=n_runder,
                                                    trenings_andel=trenings_andel,
                                                    random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-20 14:36:51,540] A new study created in memory with name: no-name-46c6e7e9-5ce8-451a-9350-dc602b994cc4


Enzymtyper Fordeling: Treningssett markert andel: 0.5
E     30
C     27
B     26
D     24
A2    20
A1     7
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:36:56,681] Trial 0 finished with value: 754.0999352852565 and parameters: {'k1': 9, 'k2': 5, 'p1': 4, 'p2': 3, 'max_iters': 7, 'pool_size': 23}. Best is trial 0 with value: 754.0999352852565.
[I 2024-03-20 14:38:15,940] Trial 1 finished with value: 917.3474555418026 and parameters: {'k1': 8, 'k2': 10, 'p1': 10, 'p2': 5, 'max_iters': 13, 'pool_size': 151}. Best is trial 0 with value: 754.0999352852565.
[I 2024-03-20 14:38:44,750] Trial 2 finished with value: 589.4485411450266 and parameters: {'k1': 2, 'k2': 3, 'p1': 3, 'p2': 2, 'max_iters': 10, 'pool_size': 190}. Best is trial 2 with value: 589.4485411450266.
[I 2024-03-20 14:38:56,349] Trial 3 finished with value: 932.4952750245341 and parameters: {'k1': 10, 'k2': 7, 'p1': 7, 'p2': 2, 'max_iters': 5, 'pool_size': 60}. Best is trial 2 with value: 589.4485411450266.
[I 2024-03-20 14:39:50,512] Trial 4 finished with value: 603.1767649501334 and parameters: {'k1': 7, 'k2': 2, 'p1': 1, 'p2': 9, 'max_iters': 19, 'pool_size':

Enzymtyper Fordeling: Treningssett markert andel: 0.7
E     42
C     37
B     36
D     33
A2    29
A1    10
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:42:50,275] Trial 0 finished with value: 665.8046670646543 and parameters: {'k1': 5, 'k2': 7, 'p1': 1, 'p2': 9, 'max_iters': 14, 'pool_size': 96}. Best is trial 0 with value: 665.8046670646543.
[I 2024-03-20 14:43:38,674] Trial 1 finished with value: 717.0407015371393 and parameters: {'k1': 6, 'k2': 7, 'p1': 3, 'p2': 5, 'max_iters': 13, 'pool_size': 117}. Best is trial 0 with value: 665.8046670646543.
[I 2024-03-20 14:43:43,237] Trial 2 finished with value: 582.3618840009984 and parameters: {'k1': 4, 'k2': 1, 'p1': 2, 'p2': 8, 'max_iters': 2, 'pool_size': 155}. Best is trial 2 with value: 582.3618840009984.
[I 2024-03-20 14:44:33,958] Trial 3 finished with value: 729.6697211898048 and parameters: {'k1': 5, 'k2': 9, 'p1': 3, 'p2': 5, 'max_iters': 17, 'pool_size': 90}. Best is trial 2 with value: 582.3618840009984.
[I 2024-03-20 14:44:35,805] Trial 4 finished with value: 634.2679401998006 and parameters: {'k1': 1, 'k2': 1, 'p1': 6, 'p2': 5, 'max_iters': 20, 'pool_size': 1

{'trenings_andel': [0.5, 0.7], 'parametere': [{'k1': 6, 'k2': 2, 'p1': 2, 'p2': 6, 'max_iters': 15, 'pool_size': 125}, {'k1': 4, 'k2': 1, 'p1': 2, 'p2': 8, 'max_iters': 2, 'pool_size': 155}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.5,537.457049,0.939780,398.957521,0.051056,269.368446,0.985024,195.877151,0.026440
1,0.7,546.198556,0.937805,398.214420,0.050920,204.641854,0.991941,138.068869,0.018532


### BHD

In [77]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # BHD
  bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 enzymtyper_umarkert,
                                                 treningssett,
                                                 test,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 gjentagelser,
                                                 n_runder,
                                                 umarkert_andel=umarkert_andel,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-21 19:01:29,298] A new study created in memory with name: no-name-0eb206e6-583e-4265-821e-e7b076620a10


Enzymtyper Fordeling: Treningssett markert andel: 1.0
E     60
C     53
B     52
D     48
A2    41
A1    14
Name: EnzymeCode, dtype: int64


[I 2024-03-21 19:01:47,425] Trial 0 finished with value: 2304.3295960403175 and parameters: {'k': 710, 'alpha': 1, 'maks_iterasjoner': 126}. Best is trial 0 with value: 2304.3295960403175.
[I 2024-03-21 19:02:02,540] Trial 1 finished with value: 2303.809883010489 and parameters: {'k': 579, 'alpha': 1, 'maks_iterasjoner': 180}. Best is trial 1 with value: 2303.809883010489.
[I 2024-03-21 19:02:15,998] Trial 2 finished with value: 2288.771338143646 and parameters: {'k': 517, 'alpha': 0, 'maks_iterasjoner': 121}. Best is trial 2 with value: 2288.771338143646.
[I 2024-03-21 19:02:27,655] Trial 3 finished with value: 2288.771338143646 and parameters: {'k': 398, 'alpha': 0, 'maks_iterasjoner': 60}. Best is trial 2 with value: 2288.771338143646.
[I 2024-03-21 19:02:42,697] Trial 4 finished with value: 2288.771338143646 and parameters: {'k': 696, 'alpha': 0, 'maks_iterasjoner': 129}. Best is trial 2 with value: 2288.771338143646.
[W 2024-03-21 19:02:47,274] Trial 5 failed with parameters: {'k'

KeyboardInterrupt: 

## Alternativ 2: Flere treningssandeler

In [71]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                          'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_design,
                                                           test_andel=test_andel,
                                                           trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)



print("Enzymtyper Fordeling: Test")
print(enzym_test)
print("Enzymtyper Fordeling: Treningssett umarkert")
print(enzym_trening_umarkert)


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Umarkert andel for bhd
umarkert_andel = 0.05

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Test
    Frekvens  Andel
E         15  22.39
B         13  19.40
C         13  19.40
D         12  17.91
A2        10  14.93
A1         4   5.97
Enzymtyper Fordeling: Treningssett umarkert
    Frekvens  Andel
D       1926  18.94
A1      1826  17.95
B       1728  16.99
C       1710  16.81
E       1631  16.04
A2      1350  13.27


### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # CoReg
  coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                   treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt,
                                                   n_runder,
                                                   umarkert_andel=False,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 16:23:34,230] A new study created in memory with name: no-name-6af24d9f-dc18-4f29-ab54-f58a5cef0b86


Enzymtyper Fordeling: Treningssett markert andel: 0.9
E     54
C     48
B     47
D     44
A2    37
A1    13
Name: EnzymeCode, dtype: int64


[I 2024-03-20 16:24:47,273] Trial 0 finished with value: 685.4226157275973 and parameters: {'k1': 2, 'k2': 9, 'p1': 1, 'p2': 2, 'max_iters': 20, 'pool_size': 137}. Best is trial 0 with value: 685.4226157275973.
[I 2024-03-20 16:25:27,526] Trial 1 finished with value: 676.5745106788513 and parameters: {'k1': 2, 'k2': 7, 'p1': 9, 'p2': 10, 'max_iters': 8, 'pool_size': 199}. Best is trial 1 with value: 676.5745106788513.
[I 2024-03-20 16:25:47,237] A new study created in memory with name: no-name-06c7cbaf-11fd-4162-8604-5afdbe9f900c


Enzymtyper Fordeling: Treningssett markert andel: 1.0
E     60
C     54
B     52
D     48
A2    41
A1    14
Name: EnzymeCode, dtype: int64


[I 2024-03-20 16:25:55,109] Trial 0 finished with value: 719.7342544291657 and parameters: {'k1': 5, 'k2': 2, 'p1': 7, 'p2': 2, 'max_iters': 4, 'pool_size': 85}. Best is trial 0 with value: 719.7342544291657.
[I 2024-03-20 16:26:29,874] Trial 1 finished with value: 760.380620936038 and parameters: {'k1': 8, 'k2': 7, 'p1': 7, 'p2': 5, 'max_iters': 11, 'pool_size': 84}. Best is trial 0 with value: 719.7342544291657.


{'trenings_andel': [0.9, 1.0], 'parametere': [{'k1': 2, 'k2': 7, 'p1': 9, 'p2': 10, 'max_iters': 8, 'pool_size': 199}, {'k1': 5, 'k2': 2, 'p1': 7, 'p2': 2, 'max_iters': 4, 'pool_size': 85}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,646.036112,0.901551,475.760757,0.062065,319.314916,0.981575,212.332258,0.027493
1,1.0,653.485454,0.899267,465.878939,0.062051,324.776339,0.980357,215.058032,0.027707


### BHD

In [72]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert)

  # BHD
  bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                               treningssett_markert,
                                               treningssett_umarkert,
                                               test,
                                               X_test,
                                               y_test,
                                               splitt,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 1.0
    Frekvens  Andel
E         60  22.30
C         54  20.07
B         52  19.33
D         48  17.84
A2        41  15.24
A1        14   5.20


[I 2024-03-21 18:59:45,762] A new study created in memory with name: no-name-df19d179-a63d-4386-95e9-4b33b914da1f
[I 2024-03-21 18:59:47,774] Trial 0 finished with value: 2170.9686384456263 and parameters: {'k': 639, 'alpha': 0.3371647851298941, 'maks_iterasjoner': 61}. Best is trial 0 with value: 2170.9686384456263.
[I 2024-03-21 18:59:49,750] Trial 1 finished with value: 2091.635869343284 and parameters: {'k': 511, 'alpha': 0.21349041223945686, 'maks_iterasjoner': 38}. Best is trial 1 with value: 2091.635869343284.
[I 2024-03-21 18:59:51,601] Trial 2 finished with value: 2483.966375752952 and parameters: {'k': 484, 'alpha': 0.5831324711168083, 'maks_iterasjoner': 5}. Best is trial 1 with value: 2091.635869343284.
[I 2024-03-21 18:59:53,497] Trial 3 finished with value: 2080.2149426699584 and parameters: {'k': 523, 'alpha': 0.12986778903345775, 'maks_iterasjoner': 87}. Best is trial 3 with value: 2080.2149426699584.
[I 2024-03-21 18:59:56,334] Trial 4 finished with value: 2493.7992022

{'trenings_andel': [1.0], 'parametere': [{'k': 523, 'alpha': 0.12986778903345775, 'maks_iterasjoner': 87}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,2189.272223,-0.13057,1954.114231,0.257903,37.074291,0.999744,5.38857,0.000926


# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzymtyper_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_umarkert.value_counts())

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 1

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
A2    7
C     4
B     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # Modelloptimalisering
  coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                     enzymtyper_treningssett_markert,
                                                     treningssett_umarkert,
                                                     treningssett,
                                                     X_test,
                                                     y_test,
                                                     splitt=splitt,
                                                     gjentagelser=gjentagelser,
                                                     n_runder=n_runder,
                                                     trenings_andel=trenings_andel,
                                                     random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-20 13:41:56,278] A new study created in memory with name: no-name-43635a3c-b79b-4d96-8b8d-bc029bfc873f


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:42:09,724] Trial 0 finished with value: 5.050093033762192 and parameters: {'k1': 8, 'k2': 9, 'p1': 4, 'p2': 10, 'max_iters': 20, 'pool_size': 12}. Best is trial 0 with value: 5.050093033762192.
[I 2024-03-20 13:43:02,888] Trial 1 finished with value: 5.073281057754931 and parameters: {'k1': 5, 'k2': 10, 'p1': 4, 'p2': 7, 'max_iters': 17, 'pool_size': 65}. Best is trial 0 with value: 5.050093033762192.
[I 2024-03-20 13:43:19,090] Trial 2 finished with value: 5.399002928636507 and parameters: {'k1': 10, 'k2': 2, 'p1': 10, 'p2': 3, 'max_iters': 18, 'pool_size': 27}. Best is trial 0 with value: 5.050093033762192.
[I 2024-03-20 13:43:38,410] Trial 3 finished with value: 5.260300386083797 and parameters: {'k1': 10, 'k2': 2, 'p1': 6, 'p2': 6, 'max_iters': 2, 'pool_size': 148}. Best is trial 0 with value: 5.050093033762192.
[I 2024-03-20 13:43:41,041] Trial 4 finished with value: 4.916827370608138 and parameters: {'k1': 9, 'k2': 7, 'p1': 5, 'p2': 3, 'max_iters': 5, 'pool_size'

{'trenings_andel': [1.0], 'parametere': [{'k1': 6, 'k2': 9, 'p1': 8, 'p2': 9, 'max_iters': 1, 'pool_size': 34}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.942628,0.297023,4.232296,0.222778,3.937273,0.388229,3.213241,0.152856


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # BHD
  bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 enzymtyper_umarkert,
                                                 treningssett,
                                                 test,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 gjentagelser,
                                                 n_runder,
                                                 umarkert_andel=umarkert_andel,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-20 13:46:46,443] A new study created in memory with name: no-name-bca6e7c0-187c-4bea-bce9-1a587c77e6b0


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:47:56,537] Trial 0 finished with value: 5.027610544334957 and parameters: {'k': 5051, 'alpha': 0.1, 'maks_iterasjoner': 146}. Best is trial 0 with value: 5.027610544334957.
[I 2024-03-20 13:48:49,062] Trial 1 finished with value: 5.027608547256884 and parameters: {'k': 2715, 'alpha': 0.1, 'maks_iterasjoner': 176}. Best is trial 1 with value: 5.027608547256884.
[I 2024-03-20 13:49:25,751] Trial 2 finished with value: 6.2877293088414685 and parameters: {'k': 1881, 'alpha': 1, 'maks_iterasjoner': 89}. Best is trial 1 with value: 5.027608547256884.
[I 2024-03-20 13:50:32,392] Trial 3 finished with value: 6.285797254057214 and parameters: {'k': 5743, 'alpha': 1, 'maks_iterasjoner': 128}. Best is trial 1 with value: 5.027608547256884.
[I 2024-03-20 13:51:00,931] Trial 4 finished with value: 9.354421732500974 and parameters: {'k': 1092, 'alpha': 10, 'maks_iterasjoner': 96}. Best is trial 1 with value: 5.027608547256884.
[I 2024-03-20 13:52:00,997] Trial 5 finished with value:

{'trenings_andel': [1.0], 'parametere': [{'k': 751, 'alpha': 0.01, 'maks_iterasjoner': 126}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,5.99464,-0.034074,4.49963,0.241266,0.072208,0.999794,0.059269,0.002705


## Alternativ 2: Flere treningssandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                          'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_uten_NIR,
                                                           test_andel=test_andel,
                                                           trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)



print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Treningssett umarkert")
print(enzym_trening_umarkert.value_counts())


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
A2    5
B     4
C     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert andel: 1.0
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # CoReg
  coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                   treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt,
                                                   n_runder,
                                                   umarkert_andel=False,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 13:54:14,192] A new study created in memory with name: no-name-ca9e0276-d8ce-43a3-a29e-868aa4cf440c


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:54:25,918] Trial 0 finished with value: 5.946402474305579 and parameters: {'k1': 10, 'k2': 6, 'p1': 8, 'p2': 3, 'max_iters': 2, 'pool_size': 154}. Best is trial 0 with value: 5.946402474305579.
[I 2024-03-20 13:56:00,212] Trial 1 finished with value: 6.0440976840206515 and parameters: {'k1': 6, 'k2': 4, 'p1': 5, 'p2': 10, 'max_iters': 20, 'pool_size': 186}. Best is trial 0 with value: 5.946402474305579.


{'trenings_andel': [1.0], 'parametere': [{'k1': 10, 'k2': 6, 'p1': 8, 'p2': 3, 'max_iters': 2, 'pool_size': 154}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.687323,0.069403,4.022689,0.196577,3.958656,0.440659,3.245401,0.158872


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # BHD
  bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                               treningssett_markert,
                                               treningssett_umarkert,
                                               test,
                                               X_test,
                                               y_test,
                                               splitt,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:56:08,112] A new study created in memory with name: no-name-6a664a9a-e232-41ee-8994-ea442fbf3316
[I 2024-03-20 13:56:56,654] Trial 0 finished with value: 13.230903344108082 and parameters: {'k': 2315, 'alpha': 10, 'maks_iterasjoner': 178}. Best is trial 0 with value: 13.230903344108082.
[I 2024-03-20 13:57:24,290] Trial 1 finished with value: 8.072900120392212 and parameters: {'k': 372, 'alpha': 1, 'maks_iterasjoner': 200}. Best is trial 1 with value: 8.072900120392212.


{'trenings_andel': [1.0], 'parametere': [{'k': 372, 'alpha': 1, 'maks_iterasjoner': 200}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,6.973406,-1.059689,5.683795,0.255547,8.019235,-1.295342,6.235015,0.266181


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzymtyper_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_umarkert.value_counts())

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 1

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
E     60
C     53
B     52
D     48
A2    41
A1    14
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
E     15
C     14
B     13
D     12
A2    10
A1     4
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
D     1926
A1    1826
B     1728
C     1710
E     1631
A2    1350
Name: EnzymeCode, dtype: int64


### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # Modelloptimalisering
  coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                    enzymtyper_treningssett_markert,
                                                    treningssett_umarkert,
                                                    treningssett,
                                                    X_test,
                                                    y_test,
                                                    splitt=splitt,
                                                    gjentagelser=gjentagelser,
                                                    n_runder=n_runder,
                                                    trenings_andel=trenings_andel,
                                                    random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-20 15:52:44,688] A new study created in memory with name: no-name-c5592913-f6af-46e8-8ffa-00b8151b5ed4


Enzymtyper Fordeling: Treningssett markert andel: 1.0
E     60
C     53
B     52
D     48
A2    41
A1    14
Name: EnzymeCode, dtype: int64


[W 2024-03-20 15:52:52,884] Trial 0 failed with parameters: {'k1': 9, 'k2': 7, 'p1': 1, 'p2': 2, 'max_iters': 6, 'pool_size': 192} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/content/moduler_optimalisering_evaluering_semiveiledet.py", line 276, in objective
    coreg.fit(X_trening_fold_markert_transformert,
  File "/usr/local/lib/python3.10/dist-packages/LAMDA_SSL/Algorithm/Regression/CoReg.py", line 79, in fit
    h_temp.predict(L_X[idx_o].reshape(1, -1))) ** 2
  File "/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_regression.py", line 236, in predict
    neigh_ind = self.kneighbors(X, return_distance=False)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_base.py", line 784, in kneighbors
    check_is_fitted(self)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/u

KeyboardInterrupt: 

### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # BHD
  bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 enzymtyper_umarkert,
                                                 treningssett,
                                                 test,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 gjentagelser,
                                                 n_runder,
                                                 umarkert_andel=umarkert_andel,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-20 14:00:21,938] A new study created in memory with name: no-name-91250566-f45f-4c07-a896-bf4b2b4e11ea


Enzymtyper Fordeling: Treningssett markert andel: 1.0
C     14
B     14
D     11
A2     5
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:00:30,633] Trial 0 finished with value: 7.0477430036266195 and parameters: {'k': 2322, 'alpha': 1, 'maks_iterasjoner': 144}. Best is trial 0 with value: 7.0477430036266195.
[I 2024-03-20 14:00:36,993] Trial 1 finished with value: 9.928171835120885 and parameters: {'k': 1405, 'alpha': 10, 'maks_iterasjoner': 113}. Best is trial 0 with value: 7.0477430036266195.
[I 2024-03-20 14:00:43,501] Trial 2 finished with value: 7.4522799199520735 and parameters: {'k': 2582, 'alpha': 1, 'maks_iterasjoner': 3}. Best is trial 0 with value: 7.0477430036266195.
[I 2024-03-20 14:00:50,428] Trial 3 finished with value: 5.085764895284852 and parameters: {'k': 1909, 'alpha': 0.1, 'maks_iterasjoner': 93}. Best is trial 3 with value: 5.085764895284852.
[I 2024-03-20 14:00:59,400] Trial 4 finished with value: 9.92754531897237 and parameters: {'k': 2198, 'alpha': 10, 'maks_iterasjoner': 147}. Best is trial 3 with value: 5.085764895284852.
[I 2024-03-20 14:01:02,658] Trial 5 finished with value

{'trenings_andel': [1.0], 'parametere': [{'k': 490, 'alpha': 0.01, 'maks_iterasjoner': 21}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.988597,-0.204528,4.089666,0.206132,0.091019,0.999674,0.066162,0.003041


## Alternativ 2: Flere treningssandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                          'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_design_uten_NIR,
                                                           test_andel=test_andel,
                                                           trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)



print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Treningssett umarkert")
print(enzym_trening_umarkert.value_counts())


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
E     15
B     13
C     13
D     12
A2    10
A1     4
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert
D     1926
A1    1826
B     1728
C     1710
E     1631
A2    1350
Name: EnzymeCode, dtype: int64


### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # CoReg
  coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                   treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt,
                                                   n_runder,
                                                   umarkert_andel=False,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 15:53:01,605] A new study created in memory with name: no-name-d08ab546-a594-4b92-b2ad-2bf98f0fa81f


Enzymtyper Fordeling: Treningssett markert andel: 1.0
E     60
C     54
B     52
D     48
A2    41
A1    14
Name: EnzymeCode, dtype: int64


[I 2024-03-20 15:53:14,966] Trial 0 finished with value: 1286.9719016355389 and parameters: {'k1': 9, 'k2': 8, 'p1': 4, 'p2': 3, 'max_iters': 11, 'pool_size': 27}. Best is trial 0 with value: 1286.9719016355389.
[I 2024-03-20 15:54:08,287] Trial 1 finished with value: 1355.1537948613854 and parameters: {'k1': 8, 'k2': 2, 'p1': 1, 'p2': 1, 'max_iters': 13, 'pool_size': 160}. Best is trial 0 with value: 1286.9719016355389.


{'trenings_andel': [1.0], 'parametere': [{'k1': 9, 'k2': 8, 'p1': 4, 'p2': 3, 'max_iters': 11, 'pool_size': 27}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,649.19508,0.900586,502.535489,0.067652,835.132393,0.870115,636.376849,0.086558


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # BHD
  bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                               treningssett_markert,
                                               treningssett_umarkert,
                                               test,
                                               X_test,
                                               y_test,
                                               splitt,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:02:33,802] A new study created in memory with name: no-name-2637f184-92b8-481b-b30b-a576cf014d62
[I 2024-03-20 14:03:06,355] Trial 0 finished with value: 11.875057535216705 and parameters: {'k': 1786, 'alpha': 1, 'maks_iterasjoner': 2}. Best is trial 0 with value: 11.875057535216705.
[I 2024-03-20 14:04:00,607] Trial 1 finished with value: 6.394147053894678 and parameters: {'k': 2593, 'alpha': 0.01, 'maks_iterasjoner': 162}. Best is trial 1 with value: 6.394147053894678.


{'trenings_andel': [1.0], 'parametere': [{'k': 2593, 'alpha': 0.01, 'maks_iterasjoner': 162}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.95906,-0.041623,4.439962,0.21455,0.173033,0.998931,0.141838,0.006167
